## EXPERIMENT 1

In [1]:
## Load libraries

import numpy as np
import cv2
from matplotlib import pyplot as plt
import torch
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import normalize
import sys
sns.set(style="white") 
%matplotlib inline
sys.path.append('../../utils') 
from MagFace_utils.MagFace_funcs import *
from Data_proc_utils.Data_proc_funcs import *

### MagFace Results

This notebook loads the feature vectors from MagFace and run them through the results metrics:
FNIR, FPIR, FND, FPD, and GARBE

### Load data

In [2]:
# Get all features from feature lists from magface model
feature_list_children = '../../data/feat_img_children_full.list'
feature_list_adults = '../../data/feat_img_adults_full.list'

image_names_c, ids_c, num_ids_c, norm_feats_c = load_magface_vectors(feature_list_children)
image_names_a, ids_a, num_ids_a, norm_feats_a = load_magface_vectors(feature_list_adults)

# Similarity matrices from magface - all
sim_mat_c = np.dot(norm_feats_c, norm_feats_c.T)
sim_mat_a = np.dot(norm_feats_a, norm_feats_a.T)

# Dataframes with info, removes names not in magface results
children_all = pd.read_csv('../../data/YLFW_full_info_excluding_adults.csv')
children_all = children_all[children_all.image_name.isin(image_names_c)]
adults_all = pd.read_csv('../../data/RFW_full_info_excluding_children.csv')
adults_all = adults_all[adults_all.image_name.isin(image_names_a)] # OBS, this operation because some magface images has not been through all adults images. 





### GET METRICS 10 TIMES

In [7]:
random_states = [3]#[1,2,3,4,5,6,7,8,9,10]
FNIR_c_list=[]
FNIR_a_list=[]
FPIR_c_list=[]
FPIR_a_list=[]
FPD_list=[]
FND_list=[]
GARBE_list=[]

for random_state_i in random_states:
    
    ### Load children and adults balanced data ###
    children_balanced_df_i = balance_child_data(children_all, print_stats=False, random_state=random_state_i)
    adults_balanced_df_i = balance_adults_data_enrolled(children_balanced_df_i, adults_all, print_stats=False, random_state=random_state_i)
    
    ### Enrolled similarity matrices ###
    # Get enrolled similarity matrix - children
    c_mates = children_balanced_df_i.groupby("identity_name").agg({'identity_name': ['count']})
    enrolled_identity_names_c = c_mates[c_mates[('identity_name', 'count')] > 1].index
    enrolled_image_names_c = list(children_balanced_df_i[children_balanced_df_i["identity_name"].isin(enrolled_identity_names_c)].image_name)
    non_enrolled_identity_names_c = c_mates[c_mates[('identity_name', 'count')] == 1].index
    non_enrolled_image_names_c = list(children_balanced_df_i[children_balanced_df_i["identity_name"].isin(non_enrolled_identity_names_c)].image_name)


    image_names_c_enrolled, ids_c_enrolled, num_ids_c_enrolled, norm_feats_c_enrolled = load_enrolled_magface_vectors(feature_list_children, enrolled_image_names_c, canonical=False, df_c_can=None)
    sim_mat_c_enrolled = np.dot(norm_feats_c_enrolled, norm_feats_c_enrolled.T)

    # Get enrolled similarity matrix - adults
    a_mates = adults_balanced_df_i.groupby("identity_name").agg({'identity_name': ['count']})
    enrolled_identity_names_a = a_mates[a_mates[('identity_name', 'count')] > 1].index
    enrolled_image_names_a = list(adults_balanced_df_i[adults_balanced_df_i["identity_name"].isin(enrolled_identity_names_a)].image_name)
    non_enrolled_identity_names_a = a_mates[a_mates[('identity_name', 'count')] == 1].index
    non_enrolled_image_names_a = list(adults_balanced_df_i[adults_balanced_df_i["identity_name"].isin(non_enrolled_identity_names_a)].image_name)


    image_names_a_enrolled, ids_a_enrolled, num_ids_a_enrolled, norm_feats_a_enrolled = load_enrolled_magface_vectors(feature_list_adults, enrolled_image_names_a, canonical=False, df_c_can=None)
    sim_mat_a_enrolled = np.dot(norm_feats_a_enrolled, norm_feats_a_enrolled.T)
    
    
    ### Non-enrolled similarity matrices ###
    # Get indices of all feature id elements that are non-enrolled ids 
    indices_c_non_enrolled = [image_names_c.index(name) for name in non_enrolled_image_names_c]
    indices_a_non_enrolled = [image_names_a.index(name) for name in non_enrolled_image_names_a]

    # Extract corresponding rows from the similarity matrix
    sim_mat_c_non_enrolled = sim_mat_c[indices_c_non_enrolled]
    sim_mat_a_non_enrolled = sim_mat_a[indices_a_non_enrolled]

    ### DET THINGS ###
    thold = 0.5
    
    ### Evaluation metrics ###
    # FNIR
    FNIR_c = compute_fnir_utils(sim_mat_c_enrolled, enrolled_image_names_c, ids_c, num_ids_c, thold=thold)    
    FNIR_a = compute_fnir_utils(sim_mat_a_enrolled, enrolled_image_names_a, ids_a, num_ids_a, thold=thold)
    # FPIR
    FPIR_c = compute_fpir_utils(sim_mat_c_non_enrolled, non_enrolled_image_names_c, ids_c, num_ids_c, thold=thold)
    FPIR_a = compute_fpir_utils(sim_mat_a_non_enrolled, non_enrolled_image_names_a, ids_a, num_ids_a, thold=thold)
    
    
    # OBS maybe compute Garbe outside funtion to choose a good alpha?
    alpha_garbe = 0.5
    FPD_i, FND_i, GARBE_i = GARBE(FNIR_c, FNIR_a, FPIR_c, FPIR_a, alpha=alpha_garbe)
    
    FNIR_c_list.append(FNIR_c)
    FNIR_a_list.append(FNIR_a)
    FPIR_c_list.append(FPIR_c)
    FPIR_a_list.append(FPIR_a)
    FPD_list.append(FPD_i)
    FND_list.append(FND_i)
    GARBE_list.append(GARBE_i)
    print("done")

# Create a DataFrame from the lists
data = {'Iteration': random_states,'FNIR_c': FNIR_c_list, 'FNIR_a': FNIR_a_list, "FPIR_c": FPIR_c_list, "FPIR_a": FPIR_a_list, "FPD": FPD_list, "FND": FND_list, "GARBE": GARBE_list} 
df_all_results = pd.DataFrame(data)




FPD result:  1.0
FND result:  0.8003787527847595
GARBE result MagFace:  0.9001893763923797
done


In [8]:
df_all_results

,Iteration,FNIR_c,FNIR_a,FPIR_c,FPIR_a,FPD,FND,GARBE
0,3,0.742498,0.927684,1.0,1.0,1.0,0.800379,0.900189


In [4]:
len(non_enrolled_image_names_a)

824

In [5]:
FPIR_c = compute_fpir_utils(sim_mat_c_non_enrolled, non_enrolled_image_names_c, ids_c, num_ids_c, thold=0.9)
print(FPIR_c)
FPIR_a = compute_fpir_utils(sim_mat_a_non_enrolled, non_enrolled_image_names_a, ids_a, num_ids_a, thold=0.5)
print(FPIR_a)

    

1.0
1.0
